# IMPORT

In [50]:
# Geometry packages
import geocoder

# Web Scraping
import requests
from bs4 import BeautifulSoup

# Data Analysis
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.cluster import KMeans

# Data Visualization
import folium # map rendering library
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

# Read Table into DataFrame

In [51]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs = pd.read_html(url)
df = dfs[0]
# remove Not assigned Borough
df = df[df['Borough'] != "Not assigned"]
# assign Borough tco NaN Neighbourhood
for i in range(len(df)):
    if df.iloc[i,2] == 'Not assigned':
        df.iloc[i,2] = df.iloc[i,1]
# combine areas with the same postal code
functions = {'Borough': 'sum', 'Neighbourhood': lambda column: ",".join(column)}
df_clean = df.groupby('Postal Code', as_index=False).aggregate(functions).reindex(columns=df.columns)
df_clean.head(10)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [52]:
df_clean.shape

(103, 3)

# Get the Coordinates for Neighborhoods

In [53]:
# latitudes, longitudes = [], []
# for postal_code in df_clean['Postal Code']:
#     lat_lng_coords = None
#     while(lat_lng_coords is None):
#       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#       lat_lng_coords = g.latlng

#     latitudes.append(lat_lng_coords[0])
#     longitudes.append(lat_lng_coords[1])

# df_clean['Latitude'] = latitudes
# df_clean['Longitudes'] = longitudes
# df_clean.head(10)

In [54]:
url = 'https://cocl.us/Geospatial_data'
df_pos = pd.read_csv(url)
df = df_clean.merge(df_pos, how='left', on='Postal Code')
df.head(10)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [55]:
df.shape

(103, 5)

# Clustering Neighborhoods in Toronto

## Select only boroughs that contain the word "Toronto"

In [56]:
df_toronto_old = df[df['Borough'].str.contains('Toronto', regex=False)]
df_toronto = df_toronto_old.rename(columns={'Neighbourhood': 'Neighborhood'})
df_toronto.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
47,M4S,Central Toronto,Davisville,43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


## Define Foursquare Credentials and Version

## Define a function that gets all the venues in a neighborhood

In [58]:
def getNearbyVenues(names, codes, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, code, lat, lng in zip(names, codes, latitudes, longitudes):
        print(code, name, sep=':')
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            code,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Create a new dataframe that contains all venues

In [59]:
df_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                            codes=df_toronto['Postal Code'],
                            latitudes=df_toronto['Latitude'],
                            longitudes=df_toronto['Longitude']
                                  )

M4E:The Beaches
M4K:The Danforth West, Riverdale
M4L:India Bazaar, The Beaches West
M4M:Studio District
M4N:Lawrence Park
M4P:Davisville North
M4R:North Toronto West, Lawrence Park
M4S:Davisville
M4T:Moore Park, Summerhill East
M4V:Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
M4W:Rosedale
M4X:St. James Town, Cabbagetown
M4Y:Church and Wellesley
M5A:Regent Park, Harbourfront
M5B:Garden District, Ryerson
M5C:St. James Town
M5E:Berczy Park
M5G:Central Bay Street
M5H:Richmond, Adelaide, King
M5J:Harbourfront East, Union Station, Toronto Islands
M5K:Toronto Dominion Centre, Design Exchange
M5L:Commerce Court, Victoria Hotel
M5N:Roselawn
M5P:Forest Hill North & West, Forest Hill Road Park
M5R:The Annex, North Midtown, Yorkville
M5S:University of Toronto, Harbord
M5T:Kensington Market, Chinatown, Grange Park
M5V:CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
M5W:Stn A PO Boxes
M5X:First Canadian Place, Undergroun

In [60]:
print(df_venues.shape)
df_venues.head(10)

(1610, 8)


,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M4E,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,M4E,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,M4E,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,M4E,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4E,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant
5,M4K,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant
6,M4K,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop
7,M4K,"The Danforth West, Riverdale",43.679557,-79.352188,Cafe Fiorentina,43.677743,-79.350115,Italian Restaurant
8,M4K,"The Danforth West, Riverdale",43.679557,-79.352188,Mezes,43.677962,-79.350196,Greek Restaurant
9,M4K,"The Danforth West, Riverdale",43.679557,-79.352188,La Diperie,43.677702,-79.352265,Ice Cream Shop


## Check how many venues are in each neighborhood

In [61]:
df_venues.groupby('Neighborhood').count()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Berczy Park,59,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16,16,16,16,16,16,16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17,17
Central Bay Street,61,61,61,61,61,61,61
Christie,16,16,16,16,16,16,16
Church and Wellesley,78,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100
Davisville,33,33,33,33,33,33,33


In [62]:
print('There are {} uniques categories.'.format(len(df_venues['Venue Category'].unique())))

There are 235 uniques categories.


## Analyze Each Neighborhood

In [63]:
# one hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_onehot.insert(0, 'Neighborhood_Name', df_venues['Neighborhood'])

df_onehot.head()

,Neighborhood_Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [64]:
df_onehot.shape

(1610, 236)

## Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [65]:
df_grouped = df_onehot.groupby('Neighborhood_Name').mean().reset_index()
df_grouped.head(10)

,Neighborhood_Name,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.016949,0.000000,0.0,0.00,0.0000,0.0000,0.0,0.0000,0.016949,0.033898,0.0,0.000000,0.0,0.016949,0.016949,0.0,0.033898,0.0,0.0,0.016949,0.0,0.000000,0.000000,0.000000,0.0,0.016949,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.016949,0.016949,0.0,0.0,0.0000,0.000000,0.033898,0.0,0.0,0.0,0.000000,0.016949,0.050847,0.084746,0.0,0.0,0.0,0.0,0.0,0.016949,0.000000,0.016949,0.000000,0.0,0.0,0.016949,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.016949,0.000000,0.000000,0.000000,0.0,0.000000,0.016949,0.0,0.000000,0.000000,0.0,0.000000,0.033898,0.000000,0.0,

## Find the top 10 most common venues for each neighborhood

In [66]:
def return_most_common_venues(row, num_top_venues):
    """
    This function sorts the venues in descending order
    """
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### create the new dataframe and display the top 10 venues for each neighborhood.

In [67]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_grouped['Neighborhood_Name']

for ind in np.arange(df_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Park,Clothing Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Furniture / Home Store,Burrito Place,Restaurant,Stadium,Italian Restaurant,Intersection,Bar
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Garden,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Farmers Market
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Airport Food Court,Airport Gate,Boat or Ferry,Boutique,Coffee Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Thai Restaurant,Bubble Tea Shop,Burger Joint,Yoga Studio


In [68]:
neighborhoods_venues_sorted.shape

(39, 11)

## Cluster Neighborhoods

### Run k-means to cluster the neighborhood into 5 clusters.

In [69]:
# set number of clusters
kclusters = 5

df_grouped_clustering = df_grouped.drop('Neighborhood_Name', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_grouped_clustering)

# check cluster labels generated for each row in the dataframe
np.unique(kmeans.labels_, return_counts=True)

(array([0, 1, 2, 3, 4]), array([11,  2, 23,  2,  1], dtype=int64))

### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [70]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
df_merged = df_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Asian Restaurant,Pub,Trail,Neighborhood,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Japanese Restaurant
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Fast Food Restaurant,Park,Brewery,Sandwich Place,Board Shop,Burrito Place,Restaurant,Italian Restaurant,Fish & Chips Shop,Steakhouse
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Yoga Studio,Diner,Italian Restaurant,Bookstore
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Dim Sum Restaurant,Park,Swim School,Bus Line,Yoga Studio,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop


### Visualize the resulting clusters

In [73]:
latitude = np.average(df_merged['Latitude'])
longitude = np.average(df_merged['Longitude'])
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1 

In [78]:
df_merged.loc[df_merged['Cluster Labels'] == 0, df_merged.columns[[0,1,2] + list(range(6, df_merged.shape[1]))]]

,Postal Code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,Health Food Store,Asian Restaurant,Pub,Trail,Neighborhood,Yoga Studio,Dog Run,Diner,Discount Store,Distribution Center
41,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Japanese Restaurant
42,M4L,East Toronto,"India Bazaar, The Beaches West",Fast Food Restaurant,Park,Brewery,Sandwich Place,Board Shop,Burrito Place,Restaurant,Italian Restaurant,Fish & Chips Shop,Steakhouse
47,M4S,Central Toronto,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Gym,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Park,Brewery
49,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Bagel Shop,Light Rail Station,Fried Chicken Joint,Restaurant,Bank,Supermarket
51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",Coffee Shop,Pub,Italian Restaurant,Bakery,Pharmacy,Pizza Place,Restaurant,Café,Yoga Studio,Snack Place
65,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,Pub,Indian Restaurant,Liquor Store,BBQ Joint,History Museum,Burger Joint,Pizza Place
66,M5S,Downtown Toronto,"University of Toronto, Harbord",Café,Bookstore,Bar,Italian Restaurant,Japanese Restaurant,Bakery,Dessert Shop,Pub,College Arts Building,Sushi Restaurant
75,M6G,Downtown Toronto,Christie,Grocery Store,Café,Park,Nightclub,Candy Store,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Coffee Shop
76,M6H,West Toronto,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Music Venue,Middle Eastern Restaurant,Supermarket,Bar,Café,Pool,Bank,Grocery Store


### Cluster 2

In [79]:
df_merged.loc[df_merged['Cluster Labels'] == 1, df_merged.columns[[0,1,2] + list(range(6, df_merged.shape[1]))]]

,Postal Code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Trail,Tennis Court,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop
50,M4W,Downtown Toronto,Rosedale,Park,Playground,Trail,Department Store,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop,Doner Restaurant


### Cluster 3

In [80]:
df_merged.loc[df_merged['Cluster Labels'] == 2, df_merged.columns[[0,1,2] + list(range(6, df_merged.shape[1]))]]

,Postal Code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,M4M,East Toronto,Studio District,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Café,Yoga Studio,Diner,Italian Restaurant,Bookstore
45,M4P,Central Toronto,Davisville North,Park,Food & Drink Shop,Sandwich Place,Hotel,Department Store,Breakfast Spot,Pizza Place,Eastern European Restaurant,Donut Shop,Doner Restaurant
46,M4R,Central Toronto,"North Toronto West, Lawrence Park",Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Chinese Restaurant,Fast Food Restaurant,Spa,Salon / Barbershop,Mexican Restaurant,Diner
52,M4Y,Downtown Toronto,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Fast Food Restaurant,Gay Bar,Café,Pub,Yoga Studio,Hotel
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",Coffee Shop,Bakery,Café,Park,Breakfast Spot,Pub,Theater,Yoga Studio,Cosmetics Shop,Shoe Store
54,M5B,Downtown Toronto,"Garden District, Ryerson",Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Middle Eastern Restaurant,Hotel,Bubble Tea Shop,Lingerie Store,Diner
55,M5C,Downtown Toronto,St. James Town,Coffee Shop,Café,Gastropub,American Restaurant,Cocktail Bar,Gym,Moroccan Restaurant,Clothing Store,Seafood Restaurant,Hotel
56,M5E,Downtown Toronto,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Beer Bar,Farmers Market,Restaurant,Seafood Restaurant,Park,Clothing Store
57,M5G,Downtown Toronto,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Japanese Restaurant,Salad Place,Thai Restaurant,Bubble Tea Shop,Burger Joint,Yoga Studio
58,M5H,Downtown Toronto,"Richmond, Adelaide, King",Coffee Shop,Café,Restaurant,Hotel,Gym,Clothing Store,Thai Restaurant,Deli / Bodega,Concert Hall,Sushi Restaurant


### Cluster 4

In [81]:
df_merged.loc[df_merged['Cluster Labels'] == 3, df_merged.columns[[0,1,2] + list(range(6, df_merged.shape[1]))]]

,Postal Code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M4N,Central Toronto,Lawrence Park,Dim Sum Restaurant,Park,Swim School,Bus Line,Yoga Studio,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop
64,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",Park,Trail,Jewelry Store,Sushi Restaurant,Bus Line,Dim Sum Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


### Cluster 5 

In [82]:
df_merged.loc[df_merged['Cluster Labels'] == 4, df_merged.columns[[0,1,2] + list(range(6, df_merged.shape[1]))]]

,Postal Code,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,M5N,Central Toronto,Roselawn,Garden,Home Service,Ice Cream Shop,Dessert Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Donut Shop
